In [4]:
## 패키지 import
import pymysql

## 확인... 모듈 
print(pymysql.version_info)

## DB연동을 위한 설정 정보 
config = {              ## DB연동을 위한 환경변수
    'host': '192.168.149.1',    # 각자의 접속 DB 주소, 127.0.0.1
    'user': 'scott',
    'password': 'tiger',
    'database': 'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True
}

try:
    ## DB연동 객체 생성
    ## 1. 직접 객체 정보 입력
    # conn = pymysql.connect(host='192.168.149.1',user='scott',password='tiger',database='work',
    #                 port=3306,charset='utf8',use_default=True)
    ## 2. 사전 설정된 정보를 적용
    conn = pymysql.connect(**config)
    
    ## SQL문 실행 객체 - 커서 객체(cursor)
    cursor = conn.cursor()
    
    ## 테이블 조회
    sql = "show tables"
    cursor.execute(sql)
    tables = cursor.fetchall()
    
    ## table 유무 확인
    if tables:
        print("table 있음\n", tables)
    else:
        print("table 없음")
        
except Exception as e:
    print("DB연동 에러 발생 : ",e)
    conn.rollback()   ## insert, update, delete
    
finally:
    cursor.close()
    conn.close()
    

(1, 4, 0, 'final', 0)
table 있음
 (('goods',),)


In [8]:
### Mysql CRUD 동작
# 1. 기존의 goods 테이블을 삭제
# 2. python 코드를 이용하여 DB연결 객체를 생성
# 3. DB안에 goods라는 테이블을 생성
# create table work.goods (
#    code int primary key,
#    name varchar(20) not null,
#    su int,
#    dan int
# );
# 4. 생성되 goods테이블에 레코드 추가 작성 
#    작성시 input을 이용하여 code, name, su, dan를 입력받아서 레코드 추가
#    conn.commit  => DB적용
# 5. 전체 목록 보기(전체 목록을 출력한 후에 전체 목록 개수도 출력
# 6. 상품명 조회 (상품명을 입력 받아 해당 상품명에 따른 레코드를 출력)
# #
import pymysql

config = {              ## DB연동을 위한 환경변수
    'host': '192.168.149.1',    # 각자의 접속 DB 주소, 127.0.0.1
    'user': 'scott',
    'password': 'tiger',
    'database': 'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True
}

try:
    # 연결 객체 생성
    conn = pymysql.connect(**config)
    
    # SQL 동작을 위한 객체 cursor
    cursor = conn.cursor()
    
    # table 생성
    sql ="""create table if not exists goods(
        code int primary key,
        name varchar(30) not null,
        su int default 0,
        dan int default 0
    )"""
    cursor.execute(sql)
    conn.commit()
    
    ## 레코드 추가
    code = int(input("코드 입력 : "))
    name = input("상품명 입력 : ")
    su = int(input("상품 수량 : "))
    dan = int(input("단가 입력 : "))
    
    sql = f"insert into goods values ({code},'{name}',{su},{dan})"
    cursor.execute(sql)
    conn.commit()   
    
    ## 전체 레코드 출력
    sql = "select * from goods"
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    for row in rows:
        print("%d   %s   %d   %d" % row)
    
    print("검색 레코드 수 : ", len(rows))
    
    ## 상품명 조회
    name = input("\n조회할 상품명 입력 : ")
    sql = f"select * from goods where name like '%{name}%'"
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    if rows:
        # 레코드가 1개 이상 존재함.... 
        for row in rows:
            print(row[0], row[1], row[2], row[3])
    else:
        print("해당 상품이 없음")
        
    
except Exception as e:
    print("DB Error 발생 : ",e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()


1   냉장고   2   85000
2   세탁기   3   550000
3   전자레인지   2   35000
4   HDTV   3   1500000
검색 레코드 수 :  4
4 HDTV 3 1500000


In [2]:
### 위 문제를 이용하여 다음과 같은 메뉴로 동작하는 DB프로그램을 작성

# -----메뉴-----
# 1. 상품 등록
# 2. 상품 조회(전체)
# 3. 상품 조회(상품명으로 조회)
# 4. 상품 수정(코드로..)
# 5. 상품 삭제(코드...)
# 0. 프로그램 종료

# 조건 1> 각 기능을 함수로 구현하세요. 
# 조건 2> 반복 기능을 사용하여 구현하세요(while을 이용한 무한 반복)
#        
import pymysql
import os
import sys

config = {              ## DB연동을 위한 환경변수
    'host': '192.168.149.1',    # 각자의 접속 DB 주소, 127.0.0.1
    'user': 'scott',
    'password': 'tiger',
    'database': 'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True
}

def CreateTable():
    try:
        # 연결 객체 생성
        conn = pymysql.connect(**config)      
        # SQL 동작을 위한 객체 cursor
        cursor = conn.cursor()
        # table 생성
        sql ="""create table if not exists goods(
            code int primary key,
            name varchar(30) not null,
            su int default 0,
            dan int default 0
        )"""
        cursor.execute(sql)
        conn.commit()
    except Exception as e:
        print("table 생성 실패 : ",e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

def Insert(code,name,su,dan):
    try:
        conn = pymysql.connect(**config)      
        cursor = conn.cursor()
        sql = f"insert into goods values({code},'{name}',{su},{dan})"
        cursor.execute(sql)
        conn.commit()
    except Exception as e:
        print("Insert 실패 : ",e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()
        
def SelectAll():
    try:
        conn = pymysql.connect(**config)      
        cursor = conn.cursor()
        sql = "select * from goods"
        cursor.execute(sql)
        rows = cursor.fetchall()
        print("code\tname\t수량\t단가")
        for row in rows:
            print("{}\t{}\t{}\t{}".format(row[0],row[1],row[2],row[3]))
        print("총 상품 수량 : ",len(rows))
    except Exception as e:
        print("전체조회 실패 : ",e)
    finally:
        cursor.close()
        conn.close()


def SelectOne(name):
    try:
        conn = pymysql.connect(**config)      
        cursor = conn.cursor()
        sql = f"select * from goods where name like '%{name}%'"
        cursor.execute(sql)
        rows = cursor.fetchall()
        if rows:
            print("code\tname\t수량\t단가")
            for row in rows:
                print("{}\t{}\t{}\t{}".format(row[0],row[1],row[2],row[3]))
        else:
            print("해당 상품이 없습니다.")
        
    except Exception as e:
        print("개별 조회 실패 : ",e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()


def Update(code):
    try:
        conn = pymysql.connect(**config)      
        cursor = conn.cursor()
        name = input("변경할 상품명을 입력 : ")
        su = int(input("변경할 수량을 입력 : "))
        dan = int(input("변경할 단가 입력 : "))
        sql = f"update goods set name={name}, su={su}, dan={dan} where code={code}"
        cursor.execute(sql)
        conn.commit()
        
    except Exception as e:
        print("Update 실패 : ",e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()


def Delete(code):
    try:
        conn = pymysql.connect(**config)      
        cursor = conn.cursor()
        sql = f"delete from goods where code={code}"
        del_sel = input("정말삭제하겠습니까?(y/N) : ")
        if del_sel.lower() == 'y':
            cursor.execute(sql)
            conn.commit()
        else: 
            print("삭제를 취소합니다.")        
    except Exception as e:
        print("삭제 실패 : ",e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()


while True:
    os.system('cls')
    print("-----메뉴-----")
    print("1. 상품 등록")
    print("2. 상품 조회(전체)")
    print("3. 상품 조회(상품명으로 조회)")
    print("4. 상품 수정")
    print("5. 상품 삭제")
    print("0. 프로그램 종료")
    
    CreateTable()
    
    sel = input("메뉴를 선택하세요 : ")
    match sel:
        case '1':
            print("레코드 추가 정보를 입력하세요:")
            code = int(input("코드 입력 : "))
            name = input("상품명 입력 : ")
            su = int(input("수량 입력 : "))
            dan = int(input("단가 입력 : "))
            Insert(code, name, su, dan)
            
        case '2':
            print("전체 상품 조회")
            SelectAll()
        case '3':
            print("개별 상품 조회")
            name = input("상품명을 입력하세요 : ")
            SelectOne(name)
        case '4':
            print("상품 정보 수정")
            code = int(input("변경할 상품 코드 입력 : "))
            Update(code)
        case '5':
            print("상품 정보 삭제")
            code = int(input("삭제할 상품 코드 입력 : "))
            Delete(code)
        case '0':
            print("프로그램을 종료합니다.")
            os.system("pause")
            sys.exit(0)
            
        case _:
            print("메뉴 선택을 잘못했습니다.")
            os.system("pause")
    


-----메뉴-----
1. 상품 등록
2. 상품 조회(전체)
3. 상품 조회(상품명으로 조회)
4. 상품 수정
5. 상품 삭제
0. 프로그램 종료
프로그램을 종료합니다.


In [28]:
## 앞서 만들어진 함수를 Class 구성하세요
# 

# -----메뉴-----
# 1. 상품 등록
# 2. 상품 조회(전체)
# 3. 상품 조회(상품명|code)
#    - 상품명 : 상품명으로 조회 
#    - 코드 입력시 : code를 사용하여 조회
# 4. 상품 수정(코드로..)
# 5. 상품 삭제(코드...)
# 0. 프로그램 종료
#  
# 조건 : 전체 기능을 하나의 클래스에서 구현!!!
# # 
import pymysql
import os
import sys

config = {              ## DB연동을 위한 환경변수
    'host': '192.168.149.1',    # 각자의 접속 DB 주소, 127.0.0.1
    'user': 'scott',
    'password': 'tiger',
    'database': 'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True
}

class Product:
        
    def __init__(self,config):
        self.config = config
        #print(type(self.config))
        self.conn = pymysql.connect(**self.config)
        self.cursor = self.conn.cursor()
        print("객체 생성 확인 : ", self.conn)
        print("커서 객체 확인 : ", self.cursor)

    def createTable(self):
        sql ="""create table if not exists goods(
            code int primary key,
            name varchar(30) not null,
            su int default 0,
            dan int default 0
        )"""
        self.cursor.execute(sql)
        self.conn.commit()
    
    def Insert(self,code,name,su,dan):
        sql = f"insert into goods values({code},'{name}',{su},{dan})"
        self.cursor.execute(sql)
        self.conn.commit()        
        
x = Product(config)
x.Insert(6, '키보드', 10, 20000)


객체 생성 확인 :  <pymysql.connections.Connection object at 0x00000185425AE230>
커서 객체 확인 :  <pymysql.cursors.Cursor object at 0x00000185425AF040>
